In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.randn(3, 16)  
class GNN(nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(16, 32)
        self.conv2 = GCNConv(32, 64)
        self.fc = nn.Linear(64, 7)  

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.fc(x)
        return torch.log_softmax(x, dim=1)

model = GNN()

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def train():
    model.train()
    optimizer.zero_grad()
    out = model(x, edge_index)
    loss = criterion(out, torch.tensor([0, 1, 2]))
    loss.backward()
    optimizer.step()
    return loss.item()
for epoch in range(100):
    loss = train()
    print(f"Epoch {epoch+1}, Loss: {loss}")


Epoch 1, Loss: 1.9903744459152222
Epoch 2, Loss: 1.8398622274398804
Epoch 3, Loss: 1.6937845945358276
Epoch 4, Loss: 1.53645658493042
Epoch 5, Loss: 1.3706477880477905
Epoch 6, Loss: 1.2139993906021118
Epoch 7, Loss: 1.0889458656311035
Epoch 8, Loss: 1.006473183631897
Epoch 9, Loss: 0.9517949223518372
Epoch 10, Loss: 0.8994606137275696
Epoch 11, Loss: 0.8396268486976624
Epoch 12, Loss: 0.7815621495246887
Epoch 13, Loss: 0.7300541400909424
Epoch 14, Loss: 0.6674091219902039
Epoch 15, Loss: 0.5994443297386169
Epoch 16, Loss: 0.54183429479599
Epoch 17, Loss: 0.47953155636787415
Epoch 18, Loss: 0.41313135623931885
Epoch 19, Loss: 0.3571511507034302
Epoch 20, Loss: 0.30276694893836975
Epoch 21, Loss: 0.24863873422145844
Epoch 22, Loss: 0.20211821794509888
Epoch 23, Loss: 0.16064931452274323
Epoch 24, Loss: 0.12318105250597
Epoch 25, Loss: 0.0923989936709404
Epoch 26, Loss: 0.06862073391675949
Epoch 27, Loss: 0.05059735104441643
Epoch 28, Loss: 0.03710778057575226
Epoch 29, Loss: 0.026981592

In [19]:
import requests
from bs4 import BeautifulSoup
import networkx as nx
import matplotlib.pyplot as plt

def fetch_article_links(article_title):
    url = f'https://en.wikipedia.org/wiki/{article_title}'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find(id='content').find_all('a', href=True)
        article_links = [link['href'][6:] for link in links if link['href'].startswith('/wiki/') and ':' not in link['href']]
        return article_links
    else:
        print(f"Failed to fetch data for article '{article_title}'.")

def construct_wikipedia_graph(seed_article, depth=1, max_nodes=50):
    graph = nx.Graph()
    visited = set()
    queue = [(seed_article, 0)]

    while queue:
        current_article, current_depth = queue.pop(0)
        visited.add(current_article)

        if current_depth < depth:
            links = fetch_article_links(current_article)
            if links:
                for link in links:
                    if len(visited) < max_nodes and link not in visited:
                        graph.add_edge(current_article, link)
                        queue.append((link, current_depth + 1))
    return graph

seed_article = 'Graph_theory'  
graph = construct_wikipedia_graph(seed_article, depth=2, max_nodes=50)

plt.figure(figsize=(8, 8))
nx.draw(graph, with_labels=True, font_weight='bold')
plt.show()


ModuleNotFoundError: No module named 'bs4'

In [18]:
!conda install bs4 -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /usr/local/anaconda3

  added / updated specs:
    - bs4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.12.2      |  py310hca03da5_0         219 KB
    bs4-4.12.2                 |   py38hd3eb1b0_0           6 KB
    ------------------------------------------------------------
                                           Total:         224 KB

The following NEW packages will be INSTALLED:

  bs4                pkgs/main/noarch::bs4-4.12.2-py38hd3eb1b0_0 

The following packages will be UPDATED:

  beautifulsoup4     

In [20]:
import wikipediaapi
import networkx as nx
import matplotlib.pyplot as plt

wiki_wiki = wikipediaapi.Wikipedia('en')
def fetch_wikipedia_data(article_title):
    page = wiki_wiki.page(article_title)
    if page.exists():
        return page.text
    else:
        print(f"Page '{article_title}' does not exist.")

def construct_wikipedia_graph(seed_article, depth=1, max_nodes=50):
    visited = set()
    graph = nx.Graph()
    queue = [(seed_article, 0)]

    while queue:
        current_article, current_depth = queue.pop(0)
        visited.add(current_article)

        if current_depth < depth:
            try:
                page = wiki_wiki.page(current_article)
                links = page.links
                for title in sorted(links.keys()):
                    if len(visited) < max_nodes and title not in visited:
                        graph.add_edge(current_article, title)
                        queue.append((title, current_depth + 1))
            except wikipediaapi.WikipediaException as e:
                print(f"Error: {e}")

    return graph

seed_article = 'Graph_theory' 
wiki_data = fetch_wikipedia_data(seed_article)
graph = construct_wikipedia_graph(seed_article, depth=2, max_nodes=50)
plt.figure(figsize=(8, 8))
nx.draw(graph, with_labels=True, font_weight='bold')
plt.show()


ModuleNotFoundError: No module named 'wikipediaapi'